 Copyright (c) 2015-2020, Swiss Federal Institute of Technology (ETH Zurich)

 All rights reserved.
 
 Redistribution and use in source and binary forms, with or without
 modification, are permitted provided that the following conditions are met:
 
 * Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.
 
 * Redistributions in binary form must reproduce the above copyright notice,
   this list of conditions and the following disclaimer in the documentation
   and/or other materials provided with the distribution.
 
 * Neither the name of the copyright holder nor the names of its
   contributors may be used to endorse or promote products derived from
   this software without specific prior written permission.
 
 THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
 AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
 IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
 DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
 FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
 DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
 SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
 CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
 OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
 OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
 

# Remote backend example

Enter connection a valid configuration to demo the remote backend.

In [5]:
config = {
    "ip": "82.130.102.xxx",
    "user": "karajan",
    "port": 22,
    "key": "$KARAJAN_ACCESS_DIR/id_ed25519",
}

First, import the Remote backend.

In [6]:
import os, sys, time
sys.path.append("..")

import datapro.driver.ssh

## Instantiating and connecting

Instantiate the SSH backend. The `__repr__` method is provided to display meaningful information about the state of the connection.

In [7]:
R = datapro.driver.ssh.SSH(backend=config)
print(repr(R))

<datapro.driver.ssh.SSH at 0x12166f400 (configured, inactive)>


In [8]:
R.connect()
print(repr(R))

<datapro.driver.ssh.SSH at 0x12166f400 (configured, active)>


In [9]:
from datapro.driver._backend import result_to_print_ready_dict

def print_result(result):
    _ = None
    
    if hasattr(result, "persistent"):
        _ = result.persistent
        delattr(result, "persistent")
    
    for k, v in result_to_print_ready_dict(result).items():
        print(f"# {k} {str(''):->{78-len(k)-1}}\n{v}")

    if _:
        k = "persistent"
        print(f"# {k} {str(''):->{78-len(k)-1}}")
        for k, v in _.items():
            print(f"{k+':':<10}{v!r}")
                
        result.persistent = _

## Running commands

Running commands is very simple, and can be performed via `run` or `sudo` methods, the latter having root privileges.

In [10]:
env = R.run("echo user=$HOSTNAME:$USER shell=$SHELL tty=$TTY ssh_tty=$SSH_TTY sty=$STY pwd=$PWD")

Results are instances of `invoke.runners.Result`, which wrap the following attributes:
    
- `command`: the command,
- `exited`: return code of the command,
- `stdout`: the standard output of the command,
- `stderr`: the standard error of the command.

In [11]:
print_result(env)

# command ----------------------------------------------------------------------
echo user=$HOSTNAME:$USER shell=$SHELL tty=$TTY ssh_tty=$SSH_TTY sty=$STY pwd=$PWD
# encoding ---------------------------------------------------------------------
UTF-8
# exited -----------------------------------------------------------------------
0
# stdout -----------------------------------------------------------------------
user=pc-10666:dle shell=/bin/bash tty= ssh_tty= sty= pwd=/home/dle
# stderr -----------------------------------------------------------------------



In [12]:
ps_pipe = R.run("ps ax | grep dle && >&2 echo 'Done'")
print_result(ps_pipe)

# command ----------------------------------------------------------------------
ps ax | grep dle && >&2 echo 'Done'
# encoding ---------------------------------------------------------------------
UTF-8
# exited -----------------------------------------------------------------------
0
# stdout -----------------------------------------------------------------------
25919 ?        Ss     0:00 sshd: dle [priv]
25995 ?        S      0:00 sshd: dle@notty
26000 ?        Ss     0:00 bash -c ps ax | grep dle && >&2 echo 'Done'
26002 ?        S      0:00 grep dle
# stderr -----------------------------------------------------------------------
Done


## Disconnecting

In [13]:
R.disconnect()